In [4]:
import socket
import time
import cv2
import os
import pyrealsense2 as rs
import numpy as np

def take_picture(filename):

    # Configure depth and color streams
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

    # Start streaming
    pipeline.start(config)

    try:
        for i in range(80):
            frames = pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            color_image = np.asanyarray(color_frame.get_data())
        
        cv2.imwrite(filename, color_image)
    finally:
        # Stop streaming
        pipeline.stop()
        
def send_image(file, tcp_socket, debug = False):

    file_size = os.stat(file).st_size  
    file_info = '%s|%s|'%(file,file_size)
    if debug:
        print(file_info)
    tcp_socket.sendall(file_info.encode('utf-8'))   #第一次发送请求，不是具体内容，而是先发送数据信息  

    f = open(file,'rb')
    has_sent = 0
    while has_sent != file_size:
        data = f.read(1024)  
        tcp_socket.sendall(data)                    #发送真实数据  
        has_sent += len(data)  

    f.close()
    if debug:
        print('sended')

In [5]:
img = 'realsense' + str(int(time.time())) + '.jpg'
take_picture(img)
print('picture taken')
try:
    sk = socket.socket()
    sk.connect(('192.168.137.23', 22222))
    print('host connected')
    send_image(img, sk, debug = True)
    print('receiving result...')
    data = sk.recv(1024).decode()
    print('result:\n' + data)
finally:
    sk.close()
    print('connection closed')

picture taken
host connected
realsense1564409694.jpg|62237|
sended
receiving result...
result:
250.51494598388672|175.11533737182617|382.9859733581543|266.4535713195801|0.54945093|1.0|
90.49444913864136|289.7757911682129|137.79805183410645|350.3803253173828|0.5457444|2.0|
250.3395652770996|205.49386978149414|382.9145622253418|297.28708267211914|0.5396735|1.0|

connection closed


In [ ]:
time.time()